In [ ]:
from qiskit import *
from numpy import *
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_vector, array_to_latex
from qiskit.quantum_info import state_fidelity
from qiskit.quantum_info import *

import matplotlib.pyplot as plt
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit.providers.aer.noise import NoiseModel
import qiskit.circuit.library as qulib
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel, mixed_unitary_error
import matplotlib.pyplot as plt
import numpy as np


aer_sim = AerSimulator()
#aer_sim = Aer.get_backend('statevector_simulator')

In [ ]:
def five_qubit(p):

    qm= QuantumRegister(1,'q_main')
    qa = QuantumRegister(4,'q_ancilla')
    qs = QuantumRegister(4,'q_syndrome')

    creg= ClassicalRegister(4,'syndrome_bits')
    #creg2= ClassicalRegister(1)

    initial_state = [sqrt(8)/3, 1/3]  # Define state |q_0>
    dm_before_channel= DensityMatrix(initial_state)
    qc= QuantumCircuit(qs,qa,qm,creg)

    qc.initialize(initial_state, qm)
    qc.barrier()
    #dm2= DensityMatrix(qc)
    #dm2_main= partial_trace(dm2,[0,1,2,3,4,5,6,7])
    # encoding circuit

    qc.h(qa[0])
    qc.s(qa[0])
    qc.cy(qa[0],qm)
    qc.barrier()
    qc.h(qa[1])
    qc.cx(qa[1],qm)
    qc.barrier()
    qc.h(qa[2])
    qc.cz(qa[2],qa[1])
    qc.cz(qa[2],qa[0])
    qc.cx(qa[2],qm)
    qc.barrier()
    qc.h(qa[3])
    qc.s(qa[3])
    qc.cz(qa[3],qa[2])
    qc.cz(qa[3],qa[0])
    qc.cy(qa[3],qm)


    qc.barrier()
    dm1= DensityMatrix(qc)
    #dm_before_channel= partial_trace(dm1,[0,1,2,3])

    # Channel Starts here

    X= qi.Operator.from_label('X')
    Y= qi.Operator.from_label('Y')
    Z= qi.Operator.from_label('Z')
    I= qi.Operator.from_label('I')


    noise_ops = [sqrt(1-3*p/4)*I,sqrt(p/4)*X,sqrt(p/4)*Y,sqrt(p/4)*Z]
    channel = Kraus(noise_ops)
    qc.append(channel,qm)
    qc.append(channel,[4])
    qc.append(channel,[5])
    qc.append(channel,[6])
    qc.append(channel,[7])

    #Channel Ends

    qc.barrier()

    # Syndrome measurement circuit

    qc.h(qs)
    qc.cx(qs[0],qa[0])
    qc.cz(qs[0],qa[1])
    qc.cz(qs[0],qa[2])
    qc.cx(qs[0],qa[3])
    ###
    qc.cx(qs[1],qa[1])
    qc.cz(qs[1],qa[2])
    qc.cz(qs[1],qa[3])
    qc.cx(qs[1],qm)
    ###
    qc.cx(qs[2],qa[0])
    qc.cx(qs[2],qa[2])
    qc.cz(qs[2],qa[3])
    qc.cz(qs[2],qm)
    ###
    qc.cz(qs[3],qa[0])
    qc.cx(qs[3],qa[1])
    qc.cx(qs[3],qa[3])
    qc.cz(qs[3],qm)
    ###
    qc.barrier()
    qc.h(qs)
    qc.barrier()
    qc.measure(qs,creg)


    # Error correction circuit

    qc.x(qa[3]).c_if(creg,6)
    qc.z(qa[3]).c_if(creg,9)
    qc.y(qa[3]).c_if(creg,15)

    qc.x(qa[2]).c_if(creg,3)
    qc.z(qa[2]).c_if(creg,4)
    qc.y(qa[2]).c_if(creg,7)

    qc.x(qa[1]).c_if(creg,1)
    qc.z(qa[1]).c_if(creg,10)
    qc.y(qa[1]).c_if(creg,11)

    qc.x(qa[0]).c_if(creg,8)
    qc.z(qa[0]).c_if(creg,5)
    qc.y(qa[0]).c_if(creg,13)

    qc.x(qm).c_if(creg,12)
    qc.z(qm).c_if(creg,2)
    qc.y(qm).c_if(creg,14)


    # qc.save_density_matrix(qubits=[4,5,6,7,8])
    # Decoding Circuit
    qc.barrier()
    qc.cy(qa[3],qm)
    qc.cz(qa[3],qa[0])
    qc.cz(qa[3],qa[2])
    qc.s(qa[3])
    qc.h(qa[3])
    qc.barrier()
    qc.cx(qa[2],qm)
    qc.cz(qa[2],qa[0])
    qc.cz(qa[2],qa[1])
    qc.h(qa[2])
    qc.barrier()
    qc.cx(qa[1],qm)
    qc.h(qa[1])
    qc.barrier()
    qc.cy(qa[0],qm)
    qc.s(qa[0])
    qc.h(qa[0])
    qc.barrier()
    qc.save_density_matrix(qubits=[8])


    sim_density = AerSimulator()
    job= sim_density.run(qc)
    result = job.result().data()
    dm_after_channel= result.get('density_matrix')

    # display(dm_before_channel.draw(),dm_after_channel.draw(),qc.draw())

    return state_fidelity(dm_before_channel,dm_after_channel)
    #return state_fidelity(dm2_main,dm_after_channel)



In [ ]:
p_values = np.arange(0, 1.01,0.01 )

px = linspace(0, 1, 100)

# Calculating fidelity from theoretical formula

x = 1 - (45/8)*p**2 + (75/8)*p**3 - (45/8)*p**4 + (9/8)*p**5


fidelities_five = []

fidelities_actual = []

# Loop through p values, calculate fidelities, and store results
for p in p_values:
    fidelities_five.append(five_qubit(p))

# Create the plot
plt.figure(figsize=(8, 6))
plt.plot(p_values,fidelities_five, marker='o',linewidth=3, linestyle='-',label='Experimental fidelity of Five qubit Code')
plt.plot(px,fidelities_actual,linewidth=3, linestyle='-',label='Theoretical fidelity of Five qubit Code')
#plt.plot(p,x,linewidth=1,label='Actual fidelity by formula')

plt.xlabel('Depolarizing Error Probability (p)')
plt.ylabel('State Fidelity')
plt.title('State Fidelity vs. Depolarizing Error in Quantum Circuit')
plt.legend()
plt.grid(True)
plt.show()